# Replication Code for Deliverable 3.4

## Overview

The following code is provided for the replication of the results of Deliverable 3.4 (Extended Energy Management with Forecasting of Renewable Generation) of Horizon Europe project Research Facility 2.0 ([https://rf20.eu/](https://rf20.eu/)).

## Code Structure

The code is divided into three main parts following the structure of report D3.4:

1. **TIME SERIES FORECASTING OF RENEWABLE GENERATION**
2. **EXTENDED ENERGY MANAGEMENT** (with Model Predictive Control)
3. **LONG-TERM RENEWABLE ENERGY SCENARIO GENERATION**

## Data Availability and Replication Limitations

Exact replication of the results, particularly for Section 2 (EXTENDED ENERGY MANAGEMENT), will be difficult as the following proprietary input data was used:

1. Real load demand profile of the accelerator
2. The grid tariff structure of the accelerator electrical energy supply contract

---


# CODE

In [ ]:
# loading datasets
'KARA_PVSF_featured_extended.csv'
   # USER MUST SPECIFY: Path to SSP climate data directory
    data_path = './climate_data'  # Modify this path solardf['PV']

# 1. TIME SERIES FORECASTING OF RENEWABLE GENERATION

In [ ]:
"""
Time Series Forecasting for PV Generation using AutoML Neural Networks

This module implements automated hyperparameter tuning and model training for 
photovoltaic (PV) power generation forecasting using LSTM neural network 
architectures from the NeuralForecast library.

Author: Malik Muhammad Abdullah @ Karlsruhe Institute of Technology(Research Facility 2.0 Project)
License: European Union Public Licence (EUPL) v1.2 or later
Copyright: © 2025 Research Facility 2.0 Consortium
"""
# Copyright © 2025 Research Facility 2.0 Consortium
#
# Licensed under the EUPL, Version 1.2 or – as soon they will be approved by
# the European Commission - subsequent versions of the EUPL (the "Licence");
# You may not use this work except in compliance with the Licence.
# You may obtain a copy of the Licence at:
#
# https://joinup.ec.europa.eu/collection/eupl/eupl-text-eupl-12
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the Licence is distributed on an "AS IS" basis,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the Licence for the specific language governing permissions and
# limitations under the Licence.

# =============================================================================
# IMPORTS
# =============================================================================

import logging
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import optuna

from neuralforecast.losses.pytorch import DistributionLoss, MAE, MQLoss, RMSE
from neuralforecast.auto import AutoLSTM
  
from neuralforecast.core import NeuralForecast

# =============================================================================
# CONFIGURATION AND SETUP
# =============================================================================

# Suppress verbose logging from PyTorch Lightning and Optuna
logging.getLogger("pytorch_lightning").setLevel(logging.WARNING)
optuna.logging.set_verbosity(optuna.logging.WARNING)

# Clear CUDA cache to prevent memory issues
torch.cuda.empty_cache()

# =============================================================================
# DATA LOADING AND PREPROCESSING
# =============================================================================

def load_and_preprocess_data(filepath='KARA_PVSF_featured_extended.csv'):
    """
    Load PV generation data with engineered features and prepare for modeling.
    
    Parameters
    ----------
    filepath : str
        Path to the CSV file containing PV generation data with features
        
    Returns
    -------
    pd.DataFrame
        Preprocessed dataframe with forward-filled missing values
        
    Notes
    -----
    - Renames 'pv_duty_factor' to 'y' as the target variable
    - Drops the original 'y' column if present
    - Forward fills any missing values to maintain temporal continuity
    """
    # Load the featured dataset
    df_pv1 = pd.read_csv(filepath)
    
    # Drop original target column and rename pv_duty_factor as the new target
    df_pv1 = df_pv1.drop(['y'], axis=1)
    df_pv1 = df_pv1.rename(columns={'pv_duty_factor': 'y'})
    
    # Forward fill missing values to maintain temporal continuity
    df_pv1 = df_pv1.fillna(method='ffill')
    
    return df_pv1


# Load the data
df_pv1 = load_and_preprocess_data('KARA_PVSF_featured_extended.csv')

# =============================================================================
# FEATURE SELECTION
# =============================================================================

# Selected features based on feature importance analysis or domain knowledge
# Features include:
# - Lag features: Previous timestep values (lag_1, lag_4, lag_96, lag_672)
# - Moving averages: EMA (4, 12, 24), WMA trends (72-168 hours)
# - Rolling statistics: Mean, max, min over 3h and 5h windows
# - Momentum and trend: Differences and momentum indicators
# - Seasonal features: Z-scores and ratios at different seasonal periods
# - Irradiance features: Global tilted, clear sky (DNI, DHI, GHI)
# - Weather features: Temperature, humidity, sunshine duration
# - Solar geometry: Altitude, zenith, incidence angles

selected_feature_names = [
    # EMA (Exponential Moving Average) features
    'pv_duty_factor_ema_4', 'y_ema_4',
    'pv_duty_factor_ema_12', 'y_ema_12',
    'pv_duty_factor_ema_24', 'y_ema_24',
    
    # Lag features (autoregressive components)
    'y_lag_1', 'pv_duty_factor_lag_1',
    'y_lag_4', 'pv_duty_factor_lag_4',
    'y_lag_96', 'pv_duty_factor_lag_96',
    'y_lag_672', 'pv_duty_factor_lag_672',
    
    # Irradiance features
    'global_tilted_irradiance',
    'global_tilted_irradiance_instant',
    'clear_sky_dni', 'clear_sky_dhi', 'clear_sky_ghi',
    'clear_sky_ratio',
    
    # Rolling window statistics
    'y_rolling_mean_3h', 'pv_duty_factor_rolling_mean_3h',
    'y_rolling_max_3h', 'pv_duty_factor_rolling_max_3h',
    'y_rolling_min_3h', 'pv_duty_factor_rolling_min_3h',
    'y_rolling_mean_5h', 'pv_duty_factor_rolling_mean_5h',
    'y_rolling_max_5h', 'pv_duty_factor_rolling_max_5h',
    
    # WMA (Weighted Moving Average) trend features
    'pv_duty_factor_wma_trend_72', 'y_wma_trend_72',
    'pv_duty_factor_wma_trend_96', 'y_wma_trend_96',
    'pv_duty_factor_wma_trend_120', 'y_wma_trend_120',
    'pv_duty_factor_wma_trend_144', 'y_wma_trend_144',
    'pv_duty_factor_wma_trend_168', 'y_wma_trend_168',
    
    # EMA trend features
    'pv_duty_factor_ema_trend_19', 'y_ema_trend_19',
    'pv_duty_factor_ema_trend_21', 'y_ema_trend_21',
    'pv_duty_factor_ema_trend_24', 'y_ema_trend_24',
    'pv_duty_factor_ema_trend_48', 'y_ema_trend_48',
    'pv_duty_factor_ema_trend_72', 'y_ema_trend_72',
    
    # Momentum and difference features
    'pv_duty_factor_momentum_48', 'y_momentum_48',
    'pv_duty_factor_diff_trend_48', 'y_diff_trend_48',
    'pv_duty_factor_momentum_144', 'y_momentum_144',
    'pv_duty_factor_diff_trend_144', 'y_diff_trend_144',
    'pv_duty_factor_momentum_168', 'y_momentum_168',
    'pv_duty_factor_diff_trend_168', 'y_diff_trend_168',
    
    # Seasonal features
    'pv_duty_factor_seasonal_zscore_96', 'y_seasonal_zscore_96',
    'pv_duty_factor_seasonal_zscore_672', 'y_seasonal_zscore_672',
    'pv_duty_factor_seasonal_ratio_672', 'y_seasonal_ratio_672',
    
    # Weather and environmental features
    'Tc(C)',  # Cell temperature
    'relative_humidity_2m',
    'sunshine_duration', 'sunshine_duration.1',
    
    # Solar geometry
    'altitude', 'zenith', 'incidence',
    
    # Timestamp and target
    'ds',  # datetime
    'y'    # target variable
]

# Select only the chosen features
df_selected = df_pv1[selected_feature_names]

# =============================================================================
# TRAIN/VALIDATION/TEST SPLIT CONFIGURATION
# =============================================================================

def prepare_forecasting_dataframe(df, unique_id=1):
    """
    Prepare dataframe for NeuralForecast format with proper datetime handling.
    
    Parameters
    ----------
    df : pd.DataFrame
        Input dataframe with features and target
    unique_id : int, optional
        Identifier for the time series (default=1)
        
    Returns
    -------
    pd.DataFrame
        Formatted dataframe ready for NeuralForecast
    """
    Y_df = df.copy()
    Y_df['unique_id'] = unique_id
    Y_df['ds'] = pd.to_datetime(Y_df['ds'])
    return Y_df


# Prepare the dataframe
Y_df2 = prepare_forecasting_dataframe(df_selected)

# Calculate train/validation/test split sizes
# Using 15% for validation and 15% for test (70% for training)
n_time = len(Y_df2.ds.unique())
val_size = int(0.15 * n_time)   # 15% validation
test_size = int(0.15 * n_time)  # 15% test

# =============================================================================
# HYPERPARAMETER SEARCH CONFIGURATION
# =============================================================================

def config_autoLSTM(trial):
    """
    Define hyperparameter search space for AutoLSTM model using Optuna.
    
    Parameters
    ----------
    trial : optuna.Trial
        Optuna trial object for hyperparameter suggestion
        
    Returns
    -------
    dict
        Dictionary of hyperparameters for the LSTM model
        
    Hyperparameters
    ---------------
    input_size : int
        Lookback window size (-1 for automatic selection)
    encoder_hidden_size : int
        Number of hidden units in encoder LSTM layers
    encoder_n_layers : int
        Number of stacked LSTM layers in encoder
    context_size : int
        Size of context vector for decoder
    decoder_hidden_size : int
        Number of hidden units in decoder LSTM layers
    learning_rate : float
        Learning rate for optimizer (log-scale search)
    max_steps : int
        Maximum training steps
    batch_size : int
        Number of samples per batch
    """
    return {
        # Lookback window: -1 = automatic, or fixed values
        "input_size": trial.suggest_categorical(
            "input_size", [-1, 4, 16, 64, 96, 168]
        ),
        "inference_input_size": -1,  # Use same as training
        "h": None,  # Horizon set externally
        
        # Encoder architecture
        "encoder_hidden_size": trial.suggest_categorical(
            "encoder_hidden_size", [16, 32, 64, 128]
        ),
        "encoder_n_layers": trial.suggest_int("encoder_n_layers", 1, 4),
        
        # Context representation
        "context_size": trial.suggest_categorical("context_size", [5, 10, 50]),
        
        # Decoder architecture
        "decoder_hidden_size": trial.suggest_categorical(
            "decoder_hidden_size", [16, 32, 64, 128]
        ),
        
        # Training configuration
        "learning_rate": trial.suggest_float(
            "learning_rate", 1e-4, 1e-1, log=True
        ),
        "max_steps": trial.suggest_categorical("max_steps", [500, 10000]),
        "batch_size": trial.suggest_categorical(
            "batch_size", [32, 64, 128, 256, 512]
        ),
        
        # Reproducibility
        "random_seed": 1,
    }


# =============================================================================
# MODEL CONFIGURATION
# =============================================================================

# Forecasting horizon: 96 timesteps (15-min intervals = 24 hours ahead)
horizon = 96

# Number of Optuna hyperparameter search trials
num_samples = 25

# Initialize models list with AutoLSTM

# but can be activated by uncommenting the respective blocks
models = [
    AutoLSTM(
        h=horizon,  # Forecast horizon
        backend="optuna",  # Use Optuna for hyperparameter optimization
        loss=MQLoss(level=[80]),  # Quantile loss at 80th percentile
        valid_loss=MQLoss(level=[80]),  # Validation uses same loss
        config=config_autoLSTM,  # Custom hyperparameter search space
        num_samples=num_samples  # Number of optimization trials
    ),
    
   
]

# =============================================================================
# MODEL TRAINING AND CROSS-VALIDATION
# =============================================================================

# Initialize NeuralForecast with model configuration
nf_LSTM = NeuralForecast(
    models=models,
    freq='15min',  # Data frequency: 15-minute intervals
    local_scaler_type='standard'  # Standardize features (z-score normalization)
)

# Perform cross-validation with automatic train/val/test splitting
# This will train the model and generate predictions on validation and test sets
pred_df_LSTM = nf_LSTM.cross_validation(
    df=Y_df2,
    val_size=val_size,
    test_size=test_size,
    n_windows=None  # Use default windowing strategy
)

# =============================================================================
# CUSTOM LOSS FUNCTIONS FOR EVALUATION
# =============================================================================

class ForecastingLosses:
    """
    Comprehensive collection of forecasting evaluation metrics.
    
    This class provides deterministic loss functions commonly used in 
    time series forecasting evaluation, including scale-dependent,
    scale-independent, and relative error metrics.
    """
    
    def __init__(self):
        """Initialize PyTorch loss functions."""
        self.mse_loss = nn.MSELoss()
        self.mae_loss = nn.L1Loss()
    
    def mse(self, y_true, y_pred):
        """
        Mean Squared Error (MSE).
        
        Measures average squared difference between predictions and actuals.
        Sensitive to outliers due to squaring.
        
        Parameters
        ----------
        y_true : torch.Tensor
            Ground truth values
        y_pred : torch.Tensor
            Predicted values
            
        Returns
        -------
        torch.Tensor
            MSE loss value
        """
        return self.mse_loss(y_pred, y_true)
    
    def rmse(self, y_true, y_pred):
        """
        Root Mean Squared Error (RMSE).
        
        Square root of MSE, interpretable in the same units as the target.
        
        Parameters
        ----------
        y_true : torch.Tensor
            Ground truth values
        y_pred : torch.Tensor
            Predicted values
            
        Returns
        -------
        torch.Tensor
            RMSE loss value
        """
        mse = self.mse_loss(y_pred, y_true)
        return torch.sqrt(mse)
    
    def mae(self, y_true, y_pred):
        """
        Mean Absolute Error (MAE).
        
        Average absolute difference between predictions and actuals.
        Less sensitive to outliers than MSE/RMSE.
        
        Parameters
        ----------
        y_true : torch.Tensor
            Ground truth values
        y_pred : torch.Tensor
            Predicted values
            
        Returns
        -------
        torch.Tensor
            MAE loss value
        """
        return self.mae_loss(y_pred, y_true)
    
    def mape(self, y_true, y_pred, epsilon=1e-8):
        """
        Mean Absolute Percentage Error (MAPE).
        
        Average absolute percentage error. Scale-independent but undefined
        when true values are zero. Asymmetric (penalizes over-predictions
        more than under-predictions).
        
        Parameters
        ----------
        y_true : torch.Tensor
            Ground truth values
        y_pred : torch.Tensor
            Predicted values
        epsilon : float, optional
            Small constant to avoid division by zero (default=1e-8)
            
        Returns
        -------
        torch.Tensor
            MAPE as a percentage
        """
        ape = torch.abs((y_true - y_pred) / (y_true + epsilon))
        return torch.mean(ape) * 100
    
    def smape(self, y_true, y_pred, epsilon=1e-8):
        """
        Symmetric Mean Absolute Percentage Error (sMAPE).
        
        Symmetric variant of MAPE that treats over- and under-predictions
        equally. Bounded between 0% and 200%.
        
        Parameters
        ----------
        y_true : torch.Tensor
            Ground truth values
        y_pred : torch.Tensor
            Predicted values
        epsilon : float, optional
            Small constant to avoid division by zero (default=1e-8)
            
        Returns
        -------
        torch.Tensor
            sMAPE as a percentage
        """
        numerator = torch.abs(y_pred - y_true)
        denominator = (torch.abs(y_true) + torch.abs(y_pred)) / 2 + epsilon
        return torch.mean(numerator / denominator) * 100
    
    def mase(self, y_true, y_pred, y_train=None, seasonality=1):
        """
        Mean Absolute Scaled Error (MASE).
        
        Scales MAE by the MAE of a naive seasonal forecast on training data.
        Values < 1 indicate better performance than naive forecast.
        
        Parameters
        ----------
        y_true : torch.Tensor
            Ground truth values
        y_pred : torch.Tensor
            Predicted values
        y_train : torch.Tensor
            Training data for computing naive forecast baseline
        seasonality : int, optional
            Seasonal period for naive forecast (default=1 for non-seasonal)
            
        Returns
        -------
        torch.Tensor
            MASE value
            
        Raises
        ------
        ValueError
            If y_train is not provided
        """
        if y_train is None:
            raise ValueError("MASE requires training data for naive forecast")
        
        # Naive forecast: use value from 'seasonality' steps ago
        naive_forecast = y_train[:-seasonality]
        naive_mae = torch.mean(torch.abs(y_train[seasonality:] - naive_forecast))
        forecast_mae = torch.mean(torch.abs(y_true - y_pred))
        
        return forecast_mae / naive_mae
    
    def mqloss(self, y_true, y_pred, q=0.8):
        """
        Multiplicative Quantile Loss (MQ Loss).
        
        Asymmetric loss function that penalizes under-prediction more heavily
        when q > 0.5 (useful for risk-averse forecasting).
        
        Parameters
        ----------
        y_true : torch.Tensor
            Ground truth values, shape [batch]
        y_pred : torch.Tensor
            Predicted values, shape [batch]
        q : float, optional
            Quantile level (default=0.8 for 80th percentile)
            
        Returns
        -------
        torch.Tensor
            MQ loss value
            
        Notes
        -----
        For q=0.8, under-predictions are penalized 4x more than over-predictions.
        """
        error = y_true - y_pred
        loss = torch.max((q - 1) * error, q * error)
        return torch.mean(loss)


class QuantileLoss(nn.Module):
    """
    Multi-quantile loss for probabilistic forecasting.
    
    Computes pinball loss across multiple quantiles simultaneously,
    enabling prediction intervals and uncertainty quantification.
    """
    
    def __init__(self, quantiles=[0.1, 0.5, 0.9]):
        """
        Initialize multi-quantile loss.
        
        Parameters
        ----------
        quantiles : list of float
            Quantile levels to compute (e.g., [0.1, 0.5, 0.9] for 
            10th percentile, median, and 90th percentile)
        """
        super().__init__()
        self.quantiles = quantiles
    
    def forward(self, y_true, y_pred):
        """
        Compute average pinball loss across all quantiles.
        
        Parameters
        ----------
        y_true : torch.Tensor
            Ground truth values
        y_pred : torch.Tensor
            Predicted quantiles, shape [batch, n_quantiles]
            
        Returns
        -------
        torch.Tensor
            Average quantile loss
        """
        losses = []
        for i, q in enumerate(self.quantiles):
            errors = y_true - y_pred[:, i]
            loss = torch.max((q - 1) * errors, q * errors)
            losses.append(torch.mean(loss))
        return torch.stack(losses).mean()


class NormalizedRMSE(nn.Module):
    """
    Normalized Root Mean Squared Error (NRMSE).
    
    Scales RMSE by a measure of data spread to make it comparable
    across different datasets or variables with different scales.
    """
    
    def __init__(self, normalization_method='range', epsilon=1e-8):
        """
        Initialize NRMSE with specified normalization method.
        
        Parameters
        ----------
        normalization_method : str, optional
            Method for normalization:
            - 'range': Normalize by (max - min)
            - 'std': Normalize by standard deviation
            - 'mean': Normalize by mean absolute value
            - 'iqr': Normalize by interquartile range (Q75 - Q25)
        epsilon : float, optional
            Small constant to avoid division by zero (default=1e-8)
        """
        super().__init__()
        self.normalization_method = normalization_method
        self.epsilon = epsilon
    
    def forward(self, y_true, y_pred):
        """
        Compute normalized RMSE.
        
        Parameters
        ----------
        y_true : torch.Tensor
            Ground truth values
        y_pred : torch.Tensor
            Predicted values
            
        Returns
        -------
        torch.Tensor
            NRMSE value
            
        Raises
        ------
        ValueError
            If unknown normalization method is specified
        """
        # Compute RMSE
        mse = nn.MSELoss()(y_pred, y_true)
        rmse = torch.sqrt(mse)
        
        # Calculate normalization factor based on method
        if self.normalization_method == 'range':
            data_range = torch.max(y_true) - torch.min(y_true)
            norm_factor = data_range + self.epsilon
        
        elif self.normalization_method == 'std':
            norm_factor = torch.std(y_true) + self.epsilon
        
        elif self.normalization_method == 'mean':
            norm_factor = torch.mean(torch.abs(y_true)) + self.epsilon
        
        elif self.normalization_method == 'iqr':
            q75 = torch.quantile(y_true, 0.75)
            q25 = torch.quantile(y_true, 0.25)
            norm_factor = (q75 - q25) + self.epsilon
        
        else:
            raise ValueError(
                f"Unknown normalization method: {self.normalization_method}"
            )
        
        # Return normalized RMSE
        normalized_rmse = rmse / norm_factor
        return normalized_rmse


class NRMSE(nn.Module):
    """
    Alternative implementation of Normalized RMSE with dynamic method selection.
    """
    
    def __init__(self):
        """Initialize NRMSE module."""
        super().__init__()
    
    def forward(self, y_true, y_pred, method='range'):
        """
        Compute NRMSE with specified normalization method.
        
        Parameters
        ----------
        y_true : torch.Tensor
            Ground truth values
        y_pred : torch.Tensor
            Predicted values
        method : str, optional
            Normalization method ('range', 'mean', 'std', or any other 
            defaults to no normalization)
            
        Returns
        -------
        torch.Tensor
            NRMSE value
        """
        rmse = torch.sqrt(nn.MSELoss()(y_pred, y_true))
        
        if method == 'range':
            norm = torch.max(y_true) - torch.min(y_true)
        elif method == 'mean':
            norm = torch.mean(torch.abs(y_true))
        elif method == 'std':
            norm = torch.std(y_true)
        else:
            norm = 1.0
        
        return rmse / (norm + 1e-8)


# =============================================================================
# COMPREHENSIVE LOSS CALCULATION
# =============================================================================

def calculate_all_losses(y_true, y_pred, y_train=None):
    """
    Calculate comprehensive set of forecasting evaluation metrics.
    
    Computes all deterministic and probabilistic metrics for model evaluation,
    providing a complete assessment of forecast quality from multiple perspectives.
    
    Parameters
    ----------
    y_true : torch.Tensor
        Ground truth values
    y_pred : torch.Tensor
        Predicted values
    y_train : torch.Tensor, optional
        Training data for MASE calculation (default=None)
        
    Returns
    -------
    dict
        Dictionary containing all computed loss metrics with keys:
        - 'MSE': Mean Squared Error
        - 'RMSE': Root Mean Squared Error
        - 'MAE': Mean Absolute Error
        - 'MAPE': Mean Absolute Percentage Error
        - 'sMAPE': Symmetric Mean Absolute Percentage Error
        - 'NRMSE_range': Normalized RMSE (range normalization)
        - 'NRMSE_std': Normalized RMSE (std normalization)
        - 'MASE_24': Mean Absolute Scaled Error (24-step seasonality)
        - 'MQ_Loss_q08': Multiplicative Quantile Loss at q=0.8
        
    Notes
    -----
    All metrics are returned as Python floats (.item() called on tensors).
    MASE_24 is only computed if y_train is provided.
    """
    losses = {}
    
    # Initialize loss function objects
    base_losses = ForecastingLosses()
    nrmse_range = NormalizedRMSE(normalization_method='range')
    nrmse_std = NormalizedRMSE(normalization_method='std')
    quantile_loss = QuantileLoss(quantiles=[0.1, 0.5, 0.9])
    
    # ===== Basic deterministic losses =====
    losses['MSE'] = base_losses.mse(y_true, y_pred).item()
    losses['RMSE'] = base_losses.rmse(y_true, y_pred).item()
    losses['MAE'] = base_losses.mae(y_true, y_pred).item()
    losses['MAPE'] = base_losses.mape(y_true, y_pred).item()
    losses['sMAPE'] = base_losses.smape(y_true, y_pred).item()
    
    # ===== Normalized RMSE variants =====
    losses['NRMSE_range'] = nrmse_range(y_true, y_pred).item()
    losses['NRMSE_std'] = nrmse_std(y_true, y_pred).item()
    
    # ===== MASE (if training series available) =====
    # Using 24-step seasonality (24 hours for 15-min data = 96 steps, but using 24 for daily pattern)
    if y_train is not None:
        losses['MASE_24'] = base_losses.mase(
            y_true, y_pred, y_train, seasonality=24
        ).item()
    
    # ===== Multiplicative Quantile Loss for 80th percentile =====
    # Useful for risk-averse forecasting where under-prediction is costly
    losses['MQ_Loss_q08'] = base_losses.mqloss(
        y_true.squeeze(), 
        y_pred.squeeze(), 
        q=0.8
        ).item()
    
    return losses


# =============================================================================
# EVALUATION ON PREDICTIONS
# =============================================================================

# Convert prediction results to PyTorch tensors for loss calculation
y_true = torch.tensor(
    pred_df_LSTM[['y']].values, 
    dtype=torch.float32
)  # Actual PV generation values

y_pred = torch.tensor(
    pred_df_LSTM[['AutoLSTM-median']].values, 
    dtype=torch.float32
)  # Median predictions from AutoLSTM

# Calculate all evaluation metrics
# Note: y_train not provided, so MASE will be skipped
results = calculate_all_losses(y_true, y_pred, y_train=None)

# Display results
results


# 2. EXTENDED ENERGY MANAGEMENT(with Model Predictive Control)

In [ ]:
"""
Model Predictive Control (MPC) for Multi-Battery Energy Storage Systems

This module implements a receding-horizon optimal control strategy for managing
heterogeneous battery energy storage fleets under dynamic electricity tariffs.
The optimization jointly minimizes energy costs, peak demand charges, and battery
degradation while respecting physical constraints and solar curtailment policies.

Features:
- Multi-group battery fleet management 
- Fixed electricity tariff structure (single-tier pricing)
- Solar curtailment and power dumping logic
- Adaptive terminal SOC targets based on forecast peak severity
- Gurobi (commercial) or CLARABEL (open-source) solver support

Author: Malik Muhammad Abdullah @ Karlsruhe Institute of Technology(Research Facility 2.0 Project)
License: European Union Public Licence (EUPL) v1.2 or later
Copyright: © 2025 Research Facility 2.0 Consortium
"""

# Copyright © 2025 Research Facility 2.0 Consortium
#
# Licensed under the EUPL, Version 1.2 or – as soon they will be approved by
# the European Commission - subsequent versions of the EUPL (the "Licence");
# You may not use this work except in compliance with the Licence.
# You may obtain a copy of the Licence at:
#
# https://joinup.ec.europa.eu/collection/eupl/eupl-text-eupl-12
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the Licence is distributed on an "AS IS" basis,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the Licence for the specific language governing permissions and
# limitations under the Licence.

# =============================================================================
# IMPORTS
# =============================================================================

import os
import numpy as np
import pandas as pd
import cvxpy as cp

# =============================================================================
# SOLVER CONFIGURATION
# =============================================================================
# Note: This code supports both Gurobi (commercial) and CLARABEL (open-source).
# For Gurobi, ensure you have a valid license (academic, commercial, or WLS).
# Set your Gurobi license via environment variables or gurobipy configuration.
# For open-source alternative, set use_gurobi=False to use CLARABEL.

# =============================================================================
# BATTERY FLEET CONFIGURATION
# =============================================================================

def build_battery_fleet(battery_params):
    """
    Convert battery parameters into standardized fleet format.
    
    Supports legacy single-technology format with automatic conversion to
    fleet-based representation for consistent internal processing.
    
    Parameters
    ----------
    battery_params : dict
        Battery configuration with keys:
        - 'capacity_per_battery': float, capacity per unit (MWh)
        - 'max_power_per_battery': float, power rating per unit (MW)
        - 'efficiency': float, round-trip efficiency (charging * discharging)
        - 'soc_min': float, minimum SOC (fraction, e.g., 0.05)
        - 'soc_max': float, maximum SOC (fraction, e.g., 0.95)
        - 'battery_type': str, technology type (e.g., 'LFP', 'NMC')
        - 'n_units': int, number of battery units (default=1)
        
    Returns
    -------
    dict
        Standardized fleet specification with:
        - 'n_groups': int, number of battery groups
        - 'cap_mwh': np.ndarray, total capacity per group (MWh)
        - 'pmax_mw': np.ndarray, total power per group (MW)
        - 'eta_ch': np.ndarray, charging efficiency per group
        - 'eta_dc': np.ndarray, discharging efficiency per group
        - 'soc_min': np.ndarray, min SOC per group
        - 'soc_max': np.ndarray, max SOC per group
        - 'names': list of str, group identifiers
        
    
    
    """
    n_units = int(battery_params.get('n_units', 1))
    
    # Handle capacity units (support both MWh and kWh inputs)
    cap_per_unit = float(battery_params['capacity_per_battery'])
    if cap_per_unit < 10:  # Assume MWh if < 10
        cap_mwh_u = cap_per_unit
    else:  # Assume kWh if >= 10
        cap_mwh_u = cap_per_unit / 1000.0
    
    # Handle power units (support both MW and kW inputs)
    p_per_unit = float(battery_params['max_power_per_battery'])
    if p_per_unit < 10:  # Assume MW if < 10
        p_mw_u = p_per_unit
    else:  # Assume kW if >= 10
        p_mw_u = p_per_unit / 1000.0
    
    # Efficiency and SOC limits
    eta = float(battery_params['efficiency'])
    soc_min = float(battery_params['soc_min'])
    soc_max = float(battery_params['soc_max'])
    tech = battery_params.get('battery_type', 'BESS')
    
    # Aggregate all units into single group
    return {
        'n_groups': 1,
        'cap_mwh': np.array([n_units * cap_mwh_u]),
        'pmax_mw': np.array([n_units * p_mw_u]),
        'eta_ch': np.array([eta]),
        'eta_dc': np.array([eta]),
        'soc_min': np.array([soc_min]),
        'soc_max': np.array([soc_max]),
        'names': [f"{tech}_fleet"]
    }


# =============================================================================
# ADAPTIVE TERMINAL SOC TARGET COMPUTATION
# =============================================================================

def compute_adaptive_soc_target(net_load_forecast, theta, soc_min=0.05,
                               margin_no_peak=0.07, margin_one_peak=0.03,
                               margin_multi_peak=0.02, soc_target_cap=0.15):
    """
    Compute adaptive terminal SOC target based on forecast peak severity.
    
    Strategy:
    - No peaks expected → Higher target (more reserves for uncertainty)
    - One peak expected → Moderate target (balanced approach)
    - Multiple peaks → Lower target (aggressive discharge justified)
    
    Parameters
    ----------
    net_load_forecast : array-like
        Net load forecast over MPC horizon (MW)
    theta : float
        Peak-shaving threshold (MW)
    soc_min : float
        Hard minimum SOC enforced at each timestep
    margin_no_peak : float
        Additional margin above soc_min when no peaks detected
    margin_one_peak : float
        Additional margin when exactly one peak detected
    margin_multi_peak : float
        Additional margin when multiple peaks detected
    soc_target_cap : float
        Maximum allowable target to avoid over-conservative behavior
        
    Returns
    -------
    float
        Computed terminal SOC target (strictly > soc_min)
    """
    nl = np.asarray(net_load_forecast).ravel()
    
    # Count how many timesteps exceed threshold
    over = np.maximum(0.0, nl - theta - 1e-6)
    num_peaks = int(np.count_nonzero(over > 0))
    
    # Select margin based on peak count
    if num_peaks == 0:
        margin = margin_no_peak
    elif num_peaks == 1:
        margin = margin_one_peak
    else:
        margin = margin_multi_peak
    
    # Compute target with bounds
    x_target = np.clip(soc_min + margin, soc_min + 1e-6, soc_target_cap)
    
    return float(x_target)


# =============================================================================
# MPC OPTIMIZER CLASS
# =============================================================================

class MPCOptimizer:
    """
    Model Predictive Control optimizer for battery energy storage systems.
    
    Solves a receding-horizon optimal control problem to minimize total costs
    (energy, demand charges, degradation) while respecting system constraints.
    
    Attributes
    ----------
    N : int
        Optimization horizon length (timesteps)
    deltaT : float
        Timestep duration (hours), fixed at 0.25 for 15-min intervals
    G : int
        Number of battery groups
    cap_mwh : np.ndarray
        Total capacity per group (MWh)
    pmax_mw : np.ndarray
        Maximum power per group (MW)
    eta_ch : np.ndarray
        Charging efficiency per group
    eta_dc : np.ndarray
        Discharging efficiency per group
    soc_min : np.ndarray
        Minimum SOC per group
    soc_max : np.ndarray
        Maximum SOC per group
    c_energy : float
        Energy price (€/kWh)
    c_peak : float
        Demand charge (€/kW/year)
    use_gurobi : bool
        Whether to use Gurobi (True) or CLARABEL (False)
    gurobi_opts : dict
        Options passed to Gurobi solver
    """
    
    def __init__(self, battery_params, tariff_params, degradation_params,
                 horizon_length=96, use_gurobi=True, gurobi_opts=None):
        """
        Initialize MPC optimizer.
        
        Parameters
        ----------
        battery_params : dict
            Battery configuration (see build_battery_fleet)
        tariff_params : dict
            Simplified tariff structure with keys:
            - 'c_energy': float, energy price (€/kWh)
            - 'c_peak': float, demand charge (€/kW/year)
        degradation_params : dict
            Degradation model parameters:
            - 'rho1': float, SOC variance penalty weight
            - 'rho2': float, power cycling penalty weight
        horizon_length : int
            MPC horizon length in timesteps (default=96 for 24 hours)
        use_gurobi : bool
            Use Gurobi solver if True, else CLARABEL (default=True)
        gurobi_opts : dict, optional
            Gurobi solver options (e.g., {'TimeLimit': 30, 'Threads': 8})
        """
        self.N = horizon_length
        self.deltaT = 0.25
        
        # Parse battery fleet
        fleet = build_battery_fleet(battery_params)
        self.G = fleet['n_groups']
        self.cap_mwh = fleet['cap_mwh']
        self.pmax_mw = fleet['pmax_mw']
        self.eta_ch = fleet['eta_ch']
        self.eta_dc = fleet['eta_dc']
        self.soc_min = fleet['soc_min']
        self.soc_max = fleet['soc_max']
        self.names = fleet['names']
        
        # Simplified tariff parameters (single-tier pricing)
        self.c_energy = tariff_params['c_energy']
        self.c_peak = tariff_params['c_peak']
        
        # Degradation parameters
        self.rho1 = degradation_params['rho1']
        self.rho2 = degradation_params['rho2']
        
        # Solver configuration
        self.use_gurobi = use_gurobi
        self.gurobi_opts = gurobi_opts or {}
    
    def optimize_timestep(self, net_load_forecast, current_socs,
                         cumulative_energy, current_max_peak):
        """
        Solve MPC optimization for one timestep.
        
        Parameters
        ----------
        net_load_forecast : array-like
            Net load forecast (MW), shape (N,)
        current_socs : array-like
            Current SOC per group, shape (G,)
        cumulative_energy : float
            Cumulative energy (kWh) - unused in simplified version
        current_max_peak : float
            Current peak (kW) - unused in simplified version
            
        Returns
        -------
        dict
            Optimization results with keys:
            - 'status': str, solver status
            - 'P_grid_from': float, grid import (MW)
            - 'P_dump': float, curtailed solar (MW)
            - 'next_socs': list, updated SOC per group
            - 'P_chg': float, total charging power (MW)
            - 'P_dchg': float, total discharging power (MW)
            - 'energy_cost': float, energy cost (€)
            - 'peak_cost': float, demand charge (€)
            - 'degradation_cost': float, degradation penalty (€)
        """
        current_socs = np.asarray(current_socs, dtype=float)
        
        # Adaptive terminal SOC target
        x_target = compute_adaptive_soc_target(
            net_load_forecast, theta=1.6, soc_min=float(np.min(self.soc_min))
        )
        
        # Decision variables
        x = cp.Variable((self.G, self.N + 1))       # SOC trajectories
        P_chg = cp.Variable((self.G, self.N))       # Charging power
        P_dchg = cp.Variable((self.G, self.N))      # Discharging power
        P_grid = cp.Variable(self.N)                # Grid import
        P_dump = cp.Variable(self.N)                # Curtailed solar
        theta = cp.Parameter(nonneg=True, value=1.6)  # Peak threshold
        
        # Constraints
        constraints = [x[:, 0] == current_socs]
        
        for k in range(self.N):
            # SOC dynamics: x[k+1] = x[k] - (Δt/E) * (P_dchg/η_dc - P_chg*η_ch)
            vec_gain = self.deltaT / self.cap_mwh
            term = P_dchg[:, k] / self.eta_dc - cp.multiply(P_chg[:, k], self.eta_ch)
            
            constraints += [
                x[:, k+1] == x[:, k] - cp.multiply(vec_gain, term),
                x[:, k+1] >= self.soc_min,
                x[:, k+1] <= self.soc_max,
                P_chg[:, k] >= 0,
                P_chg[:, k] <= self.pmax_mw,
                P_dchg[:, k] >= 0,
                P_dchg[:, k] <= self.pmax_mw,
            ]
            
            # Charging from excess solar only
            constraints += [
                cp.sum(P_chg[:, k]) <= cp.maximum(0, -net_load_forecast[k])
            ]
            
            # Curtailment logic
            constraints += [
                P_dump[k] >= 0,
                P_dump[k] >= cp.maximum(0, -net_load_forecast[k]) - cp.sum(P_chg[:, k]),
                P_dump[k] <= cp.maximum(0, -net_load_forecast[k])
            ]
            
            # Grid import cap
            constraints += [
                P_grid[k] >= 0,
                P_grid[k] <= theta
            ]
            
            # Power balance
            constraints += [
                net_load_forecast[k] == P_grid[k] + cp.sum(P_dchg[:, k]) 
                                       - cp.sum(P_chg[:, k]) + P_dump[k]
            ]
        
        # Objective components
        energy_cost = (self.c_energy * 1000) * cp.sum(P_grid) * self.deltaT
        peak_cost = (self.c_peak * 1000) * theta
        soc_penalty = self.rho1 * cp.sum_squares(x)
        power_penalty = self.rho2 * (cp.sum_squares(P_chg) + cp.sum_squares(P_dchg))
        degradation_cost = soc_penalty + power_penalty
        dump_penalty = (self.c_energy * 1000) * cp.sum(P_dump) * self.deltaT
        
        obj = cp.Minimize(energy_cost + peak_cost + degradation_cost + dump_penalty)
        
        # Solve
        prob = cp.Problem(obj, constraints)
        status = "unknown"
        
        try:
            if self.use_gurobi:
                prob.solve(
                    solver=cp.GUROBI,
                    qcp=True,  # Enable quadratic constraint programming
                    reoptimize=True,
                    verbose=self.gurobi_opts.get("OutputFlag", 0) == 1,
                    **self.gurobi_opts
                )
                status = prob.status
            else:
                prob.solve(solver=cp.CLARABEL, verbose=False)
                status = prob.status
        except Exception as e:
            status = f"solver_error: {str(e)}"
            print(f"Solver error: {e}")
        
        # Extract results
        res = {'status': status}
        
        if status in ["optimal", "optimal_inaccurate"]:
            res.update({
                'P_grid_from': float(P_grid.value[0]),
                'P_dump': float(P_dump.value[0]),
                'next_socs': x.value[:, 1].tolist(),
                'P_chg': float(np.sum(P_chg.value[:, 0])),
                'P_dchg': float(np.sum(P_dchg.value[:, 0])),
                'energy_cost': float(energy_cost.value),
                'peak_cost': float(peak_cost.value),
                'degradation_cost': float(degradation_cost.value),
                'soc_penalty': float(soc_penalty.value),
                'power_penalty': float(power_penalty.value),
                'dump_penalty': float(dump_penalty.value),
            })
        else:
            res.update({
                'P_grid_from': 0.0, 'P_dump': 0.0,
                'next_socs': current_socs.tolist(),
                'P_chg': 0.0, 'P_dchg': 0.0,
                'energy_cost': 0.0, 'peak_cost': 0.0,
                'degradation_cost': 0.0, 'soc_penalty': 0.0,
                'power_penalty': 0.0, 'dump_penalty': 0.0,
            })
        
        return res


# =============================================================================
# ANNUAL SIMULATION
# =============================================================================

def run_year_simulation(df, PVSF=2, battery_params=None, tariff_params=None,
                       degradation_params=None, use_gurobi=True, gurobi_opts=None):
    """
    Run full-year MPC simulation with receding horizon control.
    
    Parameters
    ----------
    df : pd.DataFrame
        Input data with columns:
        - 'PV_generation_kW': PV generation (kW), 15-min resolution
        - 'load_demand_kW': Load demand (kW), 15-min resolution
    PVSF : float
        PV scaling factor (default=2)
    battery_params : dict, optional
        Battery configuration
    tariff_params : dict, optional
        Simplified tariff structure with 'c_energy' and 'c_peak'
    degradation_params : dict, optional
        Degradation model parameters
    use_gurobi : bool
        Use Gurobi solver if True, else CLARABEL (default=True)
    gurobi_opts : dict, optional
        Gurobi solver options
        
    Returns
    -------
    results : dict
        Simulation results with time-series data
    cumulative_energy : float
        Total annual energy (kWh)
    peak_power : float
        Annual peak demand (kW)
    tariff_params : dict
        Applied tariff parameters
    degradation_params : dict
        Applied degradation parameters
    """
    # Construct net load forecast
    solar = (PVSF * df['PV_generation_kW'].values) / 1000.0  # MW
    load = df['load_demand_kW'].values / 1000.0  # MW
    net_load = load - solar  # MW
    
    # Simulation setup
    N = 96  # Horizon length
    days = len(net_load) // N
    total_steps = days * N
    deltaT = 0.25
    
    # Default parameters
    if battery_params is None:
        battery_params = {
            'capacity_per_battery': 932 / 1000,  # MWh
            'max_power_per_battery': 0.5 * 932 / 1000,  # MW
            'efficiency': 0.986**2,
            'soc_min': 0.05, 'soc_max': 0.95,
            'battery_type': 'LFP',
            'n_units': 2
        }
    
    if tariff_params is None:
        # Simplified single-tier tariff structure
        tariff_params = {
            'c_energy': 0.40,   # €/kWh (example average rate)
            'c_peak': 80.0      # €/kW/year (example average demand charge)
        }
    
    if degradation_params is None:
        degradation_params = {'rho1': 0.25, 'rho2': 5.0}
    
    if gurobi_opts is None:
        gurobi_opts = {
            "TimeLimit": 30,           # Maximum solve time (seconds)
            "FeasibilityTol": 1e-8,    # Primal feasibility tolerance
            "OptimalityTol": 1e-6,     # Dual feasibility tolerance
            "QCPDual": 0,              # Disable dual variables for QCP
            "Threads": 0,              # Use all available cores (0 = auto)
            "OutputFlag": 0            # Suppress Gurobi console output
        }
    
    # Initialize MPC
    mpc = MPCOptimizer(
        battery_params, tariff_params, degradation_params,
        horizon_length=N, use_gurobi=use_gurobi, gurobi_opts=gurobi_opts
    )
    
    # Allocate results
    results = {
        'grid_import': np.zeros(total_steps),
        'energy_dumped': np.zeros(total_steps),
        'battery_soc': np.zeros(total_steps + 1),
        'battery_chg': np.zeros(total_steps),
        'battery_dchg': np.zeros(total_steps),
        'energy_cost': np.zeros(total_steps),
        'peak_cost': np.zeros(total_steps),
        'degradation_cost': np.zeros(total_steps),
    }
    
    # Initialize state
    current_socs = mpc.soc_min.copy()
    cumulative_energy = 0.0
    current_max_peak = 0.0
    
    # Simulation loop
    for step in range(total_steps):
        # Build forecast
        forecast_start = step
        forecast_end = min(step + N, total_steps)
        
        if forecast_end < step + N:
            forecast = np.zeros(N)
            avail = forecast_end - forecast_start
            forecast[:avail] = net_load[forecast_start:forecast_end]
        else:
            forecast = net_load[forecast_start:forecast_end]
        
        # Optimize
        step_res = mpc.optimize_timestep(forecast, current_socs,
                                        cumulative_energy, current_max_peak)
        
        # Store results
        results['grid_import'][step] = step_res['P_grid_from']
        results['energy_dumped'][step] = step_res['P_dump']
        results['battery_soc'][step] = current_socs[0]
        results['battery_chg'][step] = step_res['P_chg']
        results['battery_dchg'][step] = step_res['P_dchg']
        results['energy_cost'][step] = step_res['energy_cost']
        results['peak_cost'][step] = step_res['peak_cost']
        results['degradation_cost'][step] = step_res['degradation_cost']
        
        # Update state
        current_socs = np.array(step_res['next_socs'])
        cumulative_energy += results['grid_import'][step] * deltaT * 1000.0
        current_max_peak = max(current_max_peak, results['grid_import'][step] * 1000.0)
        
        # Progress reporting
        if (step + 1) % (24 * 4) == 0:
            day = (step + 1) // (24 * 4)
            print(f"Day {day}: Peak={current_max_peak/1000:.3f}MW, "
                  f"SOC={current_socs[0]:.3f}, Status={step_res['status']}")
    
    results['battery_soc'][-1] = current_socs[0]
    
    return results, cumulative_energy, current_max_peak, tariff_params, degradation_params


# =============================================================================
# RESULTS ANALYSIS
# =============================================================================

def analyze_results(results, cumulative_energy, annual_peak_power,
                   tariff_params, degradation_params):
    """
    Print annual cost summary.
    
    Parameters
    ----------
    results : dict
        Simulation results
    cumulative_energy : float
        Annual energy (kWh)
    annual_peak_power : float
        Annual peak (kW)
    tariff_params : dict
        Tariff parameters
    degradation_params : dict
        Degradation parameters
        
    Returns
    -------
    dict
        Cost summary
    """
    energy_cost = cumulative_energy * tariff_params['c_energy']
    peak_cost = annual_peak_power * tariff_params['c_peak']
    total_degradation = np.sum(results['degradation_cost'])
    total_cost = energy_cost + peak_cost + total_degradation
    
    print("\n" + "=" * 60)
    print("ANNUAL RESULTS")
    print("=" * 60)
    print(f"Total Energy: {cumulative_energy:,.0f} kWh")
    print(f"Peak Power: {annual_peak_power:,.1f} kW")
    print(f"Energy Cost: {energy_cost:,.0f} €")
    print(f"Peak Cost: {peak_cost:,.0f} €")
    print(f"Degradation Cost: {total_degradation:,.0f} €")
    print(f"TOTAL COST: {total_cost:,.0f} €")
    print("=" * 60)
    
    return {
        'total_energy_kWh': cumulative_energy,
        'peak_power_kW': annual_peak_power,
        'energy_cost': energy_cost,
        'peak_cost': peak_cost,
        'degradation_cost': total_degradation,
        'total_cost': total_cost
    }

# =============================================================================
# MAIN FUNCTION
# =============================================================================

if __name__ == "__main__":
    """
    Usage require user-provided PV generation data.
    
    Required input CSV with column:
    - 'PV_generation_kW': Annual PV generation (kW, 15-min intervals)
    
    Load demand profile is generated synthetically to match the PV profile length.
    """
    
    # Load PV generation data (user must provide this file)
    df_pv = solardf['PV']
    
    # Verify required column exists
    if 'PV_generation_kW' not in df_pv.columns:
        raise ValueError("Input file must contain 'PV_generation_kW' column")
    
    # Generate synthetic load demand profile
    # Characteristics: realistic industrial/commercial load pattern
    n_steps = len(df_pv)
    n_days = n_steps // 96  # 96 timesteps per day (15-min intervals)
    
    print(f"Generating synthetic load profile for {n_days} days ({n_steps} timesteps)...")
    
    # Create time index for pattern generation
    timestep_of_day = np.tile(np.arange(96), n_days)[:n_steps]  # 0-95 for each day
    day_of_year = np.repeat(np.arange(n_days), 96)[:n_steps]
    
    # Base load profile: typical daily pattern (kW)
    # Higher during business hours (08:00-18:00), lower at night
    hour_of_day = timestep_of_day / 4.0  # Convert to hours (0-24)
    base_daily_pattern = (
        500 +  # Minimum base load
        800 * np.clip(np.sin((hour_of_day - 6) * np.pi / 12), 0, 1) +  # Daytime peak
        200 * np.random.rand(n_steps)  # Random variation
    )
    
    # Seasonal variation: higher load in summer (cooling) and winter (heating)
    seasonal_factor = 1.0 + 0.3 * np.cos(2 * np.pi * day_of_year / 365 - np.pi)
    
    # Weekly pattern: lower load on weekends
    day_of_week = (day_of_year % 7)
    weekend_factor = np.where((day_of_week == 5) | (day_of_week == 6), 0.7, 1.0)
    weekend_factor = np.repeat(weekend_factor, 96)[:n_steps]
    
    # Random noise and occasional load spikes
    random_noise = np.random.normal(1.0, 0.1, n_steps)
    random_spikes = np.random.choice([1.0, 1.3], size=n_steps, p=[0.98, 0.02])
    
    # Combine all components
    load_demand_kW = (
        base_daily_pattern * 
        seasonal_factor * 
        weekend_factor * 
        random_noise * 
        random_spikes
    )
    
    # Ensure non-negative and clip to realistic range
    load_demand_kW = np.clip(load_demand_kW, 100, 3000)
    
    # Create final dataframe with both PV generation and synthetic load
    df = pd.DataFrame({
        'PV_generation_kW': df_pv['PV_generation_kW'].values,
        'load_demand_kW': load_demand_kW
    })
    
    # Print statistics
    print(f"\nSynthetic Load Profile Statistics:")
    print(f"  Mean load: {df['load_demand_kW'].mean():.1f} kW")
    print(f"  Peak load: {df['load_demand_kW'].max():.1f} kW")
    print(f"  Min load: {df['load_demand_kW'].min():.1f} kW")
    print(f"  Total energy: {df['load_demand_kW'].sum() * 0.25 / 1000:.1f} MWh")
    print(f"\nPV Generation Statistics:")
    print(f"  Mean generation: {df['PV_generation_kW'].mean():.1f} kW")
    print(f"  Peak generation: {df['PV_generation_kW'].max():.1f} kW")
    print(f"  Total energy: {df['PV_generation_kW'].sum() * 0.25 / 1000:.1f} MWh")
    
    # Configure system
    battery_params = {
        'capacity_per_battery': 0.932,  # MWh
        'max_power_per_battery': 0.466,  # MW
        'efficiency': 0.972,
        'soc_min': 0.05, 'soc_max': 0.95,
        'battery_type': 'LFP',
        'n_units': 2
    }
    
    # Simplified tariff (single-tier)
    tariff_params = {
        'c_energy': 0.40,  # €/kWh
        'c_peak': 80.0     # €/kW/year
    }
    
    # Gurobi solver options (no license credentials in code)
    gurobi_opts = {
        "TimeLimit": 30,
        "Threads": 8,
        "OutputFlag": 0
    }
    
    # Run simulation with Gurobi
    print("\nStarting MPC simulation...")
    results, cum_e, peak, tariff, degr = run_year_simulation(
        df, PVSF=2, 
        battery_params=battery_params,
        tariff_params=tariff_params,
        use_gurobi=True,
        gurobi_opts=gurobi_opts
    )
    
    # Analyze
    summary = analyze_results(results, cum_e, peak, tariff, degr)
    
    print("\nSimulation complete!")



# 3. LONG-TERM RENEWABLE ENERGY SCENARIO GENERATION

In [ ]:
"""
Climate-Driven PV Generation Scenario Analysis

This module generates probabilistic PV generation scenarios from climate projection
data (SSP pathways) using kernel density estimation and quantile sampling methods.
It performs technology sensitivity analysis to separate climate uncertainty from
technological improvements in PV panel efficiency.

Features:
- SSP climate pathway processing (GHI, temperature)
- Solar position and POA irradiance calculation (pvlib)
- KDE-based synthetic year generation
- Quantile-based scenario selection
- PV power modeling with temperature effects
- Technology sensitivity analysis (panel efficiency variations)

Author: Malik Muhammad Abdullah @ Karlsruhe Institute of Technology(Research Facility 2.0 Project)
License: European Union Public Licence (EUPL) v1.2 or later
Copyright: © 2025 Research Facility 2.0 Consortium
"""

# Copyright © 2025 Research Facility 2.0 Consortium
#
# Licensed under the EUPL, Version 1.2 or – as soon they will be approved by
# the European Commission - subsequent versions of the EUPL (the "Licence");
# You may not use this work except in compliance with the Licence.
# You may obtain a copy of the Licence at:
#
# https://joinup.ec.europa.eu/collection/eupl/eupl-text-eupl-12
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the Licence is distributed on an "AS IS" basis,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the Licence for the specific language governing permissions and
# limitations under the Licence.

# =============================================================================
# IMPORTS
# =============================================================================

import pandas as pd
import numpy as np
from pathlib import Path
from scipy.stats import gaussian_kde
import matplotlib.pyplot as plt
from pvlib import solarposition, irradiance
from pvlib.location import Location
import warnings
warnings.filterwarnings('ignore')

# =============================================================================
# CONFIGURATION
# =============================================================================

# Output directory for results
OUTPUT_DIR = Path('./scenario_results')
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Site parameters (example location - user should modify)
SITE_CONFIG = {
    'latitude': 49.0,      # Decimal degrees North
    'longitude': 8.4,      # Decimal degrees East
    'altitude': 110,       # Meters above sea level
    'timezone': 'Europe/Berlin',
    'name': 'Site_Location'
}

# PV system parameters (example configuration)
PV_CONFIG = {
    'N_PV': 800,           # Number of PV panels
    'tilt': 5,             # Panel tilt angle (degrees)
    'azimuth': 180,        # Panel azimuth (180 = south-facing)
    'module_specs': {
        'ratedpowerkw': 0.4,           # Rated power per panel (kW)
        'SurfaceAream2': 1.92,         # Panel surface area (m²)
        'panel_efficiency': 0.20,       # Panel efficiency (20%)
        'temp_coeff': -0.0039,         # Temperature coefficient (/°C)
        'ref_temp': 25,                # Reference temperature (°C)
        'ncot': 45,                    # Nominal cell operating temp (°C)
        'Degrading_Factor': 0.88       # System degradation factor
    }
}

# =============================================================================
# PV POWER CALCULATION MODEL
# =============================================================================

def calculate_pv_power_flexible(
    temperature: np.ndarray,
    gti_irradiance: np.ndarray,
    N_PV: float,
    module_specs: dict = None,
    use_rated_power: bool = True
) -> np.ndarray:
    """
    Calculate PV power output with temperature corrections.
    
    Models the relationship between incident irradiance, cell temperature,
    and electrical power output, accounting for temperature-dependent
    efficiency losses.
    
    Parameters
    ----------
    temperature : np.ndarray
        Ambient temperature (°C)
    gti_irradiance : np.ndarray
        Global tilted irradiance on panel surface (W/m²)
    N_PV : float
        Number of PV panels
    module_specs : dict, optional
        Module specifications (see PV_CONFIG)
    use_rated_power : bool, optional
        If True, use rated power method; else use area-efficiency method
        
    Returns
    -------
    np.ndarray
        PV power output (kW)
        
    Notes
    -----
    Cell temperature model: T_cell = T_amb + GTI * (NCOT - 20) / 800
    Temperature loss factor: 1 + temp_coeff * (T_cell - T_ref)
    """
    if module_specs is None:
        module_specs = PV_CONFIG['module_specs']
    
    temp_ambient = np.array(temperature)
    gti = np.array(gti_irradiance)
    
    # Calculate cell temperature from ambient and irradiance
    cell_temp = temp_ambient + (gti * (module_specs['ncot'] - 20) / 800)
    
    # Temperature-dependent efficiency correction
    temp_loss_factor = 1 + module_specs['temp_coeff'] * (cell_temp - module_specs['ref_temp'])
    
    if use_rated_power:
        # Method 1: Based on rated power (STC conditions)
        power_kw = (
            (gti / 1000.0) * N_PV * module_specs['ratedpowerkw'] *
            temp_loss_factor * module_specs['Degrading_Factor']
        )
    else:
        # Method 2: Based on panel area and efficiency
        power_kw = (
            gti * N_PV * module_specs['SurfaceAream2'] *
            module_specs['panel_efficiency'] * temp_loss_factor *
            module_specs['Degrading_Factor'] / 1000.0
        )
    
    return power_kw

# =============================================================================
# SSP CLIMATE DATA LOADING
# =============================================================================

def load_ssp_climate_data(base_path):
    """
    Load and standardize SSP climate projection data from CSV files.
    
    Expected file naming: Files containing 'SSP119', 'SSP126', 'SSP245',
    'SSP370', 'SSP460', or 'SSP585' in filename.
    
    Expected columns:
    - Datetime column (first column)
    - Column containing 'rsds' (surface downwelling shortwave radiation, W/m²)
    - Column containing 'tas' (near-surface air temperature, K)
    
    Parameters
    ----------
    base_path : str or Path
        Directory containing SSP climate data CSV files
        
    Returns
    -------
    pd.DataFrame
        Combined climate data with columns:
        - datetime: timestamp
        - year, month, hour, day_of_year: temporal indices
        - ssp: scenario identifier (e.g., 'ssp126')
        - ghi: global horizontal irradiance (W/m²)
        - temperature_C: air temperature (°C)
        
    Notes
    -----
    SSP pathways represent different climate futures:
    - SSP1-1.9, SSP1-2.6: Low emissions, Paris Agreement targets
    - SSP2-4.5: Middle-of-the-road, moderate emissions
    - SSP3-7.0, SSP5-8.5: High emissions, business-as-usual
    
    Data typically sourced from CMIP6 climate models via CORDEX or similar.
    """
    base_path = Path(base_path)
    ssp_files = sorted(list(base_path.glob('*SSP*.csv')))
    
    if len(ssp_files) == 0:
        raise FileNotFoundError(f"No SSP files found in {base_path}")
    
    print(f"\n{'='*70}")
    print(f"STEP 1: LOADING SSP CLIMATE DATA")
    print(f"{'='*70}")
    print(f"Found {len(ssp_files)} SSP files\n")
    
    all_data = []
    
    for file_path in ssp_files:
        filename = file_path.stem
        
        # Identify SSP scenario from filename
        ssp = None
        for ssp_id in ['119', '126', '245', '370', '460', '585']:
            if f'SSP{ssp_id}' in filename or f'ssp{ssp_id}' in filename.lower():
                ssp = f'ssp{ssp_id}'
                break
        
        if not ssp:
            print(f"Skipping {filename}: No recognized SSP identifier")
            continue
        
        print(f"Loading {ssp}...")
        
        try:
            df_temp = pd.read_csv(file_path)
            cols = df_temp.columns.tolist()
            
            # First column assumed to be datetime
            datetime_col = cols[0]
            
            # Find irradiance column (rsds = surface downwelling shortwave)
            rsds_col = None
            for col in cols:
                if 'rsds' in col.lower() and 'rsdscs' not in col.lower():
                    rsds_col = col
                    break
            
            # Find temperature column (tas = near-surface air temperature)
            tas_col = None
            for col in cols:
                if 'tas' in col.lower():
                    tas_col = col
                    break
            
            if not rsds_col or not tas_col:
                print(f"  ERROR: Missing required columns (rsds, tas)")
                continue
            
            # Standardize columns
            df_temp['datetime'] = pd.to_datetime(df_temp[datetime_col])
            df_temp['ssp'] = ssp
            df_temp['ghi'] = df_temp[rsds_col]  # W/m²
            df_temp['temperature_K'] = df_temp[tas_col]  # Kelvin
            df_temp['temperature_C'] = df_temp['temperature_K'] - 273.15  # Convert to Celsius
            df_temp['year'] = df_temp['datetime'].dt.year
            df_temp['month'] = df_temp['datetime'].dt.month
            df_temp['hour'] = df_temp['datetime'].dt.hour
            df_temp['day_of_year'] = df_temp['datetime'].dt.dayofyear
            
            # Select final columns
            final_cols = ['datetime', 'year', 'month', 'hour', 'day_of_year',
                         'ssp', 'ghi', 'temperature_C']
            df_clean = df_temp[final_cols].copy()
            
            print(f"  Loaded {len(df_clean):,} records")
            print(f"  Years: {df_clean['year'].min()}-{df_clean['year'].max()}")
            
            all_data.append(df_clean)
            
        except Exception as e:
            print(f"  ERROR: {e}")
            continue
    
    if len(all_data) == 0:
        raise ValueError("No valid SSP files were loaded")
    
    combined = pd.concat(all_data, ignore_index=True)
    
    print(f"\n{'='*70}")
    print(f"Total records loaded: {len(combined):,} across {len(all_data)} SSPs")
    print(f"{'='*70}\n")
    
    output_file = OUTPUT_DIR / 'step1_raw_climate_data.csv'
    combined.to_csv(output_file, index=False)
    print(f"✓ Saved: {output_file}\n")
    
    return combined

# =============================================================================
# IRRADIANCE CONVERSION TO TILTED SURFACE
# =============================================================================

def convert_to_tilted_irradiance_pvlib(df, site_config=SITE_CONFIG, pv_config=PV_CONFIG):
    """
    Convert GHI to plane-of-array (POA) irradiance using pvlib.
    
    Process:
    1. Calculate solar position (zenith, azimuth) for site and times
    2. Decompose GHI into DNI (direct) and DHI (diffuse) components
    3. Apply transposition model to calculate POA irradiance
    
    Parameters
    ----------
    df : pd.DataFrame
        Climate data with 'datetime', 'ghi', 'temperature_C'
    site_config : dict
        Site location parameters
    pv_config : dict
        PV system parameters (tilt, azimuth)
        
    Returns
    -------
    pd.DataFrame
        Input data augmented with:
        - zenith, azimuth_sun: solar position
        - dni, dhi: direct and diffuse horizontal irradiance
        - poa_global, poa_direct, poa_diffuse: plane-of-array components
        
    Notes
    -----
    Uses Erbs decomposition model (1982) for GHI → DNI/DHI separation.
    Uses isotropic sky model for transposition to tilted surface.
    """
    print(f"{'='*70}")
    print(f"STEP 2: CONVERTING TO TILTED IRRADIANCE (PVLIB)")
    print(f"{'='*70}")
    print(f"Site: {site_config['name']}")
    print(f"Location: ({site_config['latitude']:.4f}°N, {site_config['longitude']:.4f}°E)")
    print(f"Tilt: {pv_config['tilt']}°, Azimuth: {pv_config['azimuth']}°\n")
    
    location = Location(
        latitude=site_config['latitude'],
        longitude=site_config['longitude'],
        altitude=site_config['altitude'],
        tz=site_config['timezone']
    )
    
    df_work = df.copy()
    
    print("Calculating solar position...")
    solar_pos = location.get_solarposition(df_work['datetime'])
    
    zenith = solar_pos['apparent_zenith'].values
    azimuth = solar_pos['azimuth'].values
    
    df_work['zenith'] = zenith
    df_work['azimuth_sun'] = azimuth
    
    print("Decomposing GHI into DNI and DHI (Erbs model)...")
    
    # Calculate clearness index (kt)
    zenith_rad = np.radians(zenith)
    cos_zenith = np.cos(zenith_rad)
    cos_zenith = np.clip(cos_zenith, 0, 1)
    
    daytime_mask = zenith < 90
    
    # Extraterrestrial irradiance = 1367 W/m²
    df_work['kt'] = 0.0
    df_work.loc[daytime_mask, 'kt'] = np.clip(
        df_work.loc[daytime_mask, 'ghi'].values / (1367 * cos_zenith[daytime_mask] + 1e-10),
        0, 1
    )
    
    # Erbs diffuse fraction model
    kt = df_work['kt'].values
    df_work['diffuse_fraction'] = np.where(
        kt <= 0.22,
        1 - 0.09 * kt,
        np.where(
            kt <= 0.8,
            0.9511 - 0.1604 * kt + 4.388 * kt**2 - 16.638 * kt**3 + 12.336 * kt**4,
            0.165
        )
    )
    
    # Calculate DHI and DNI
    df_work['dhi'] = df_work['ghi'] * df_work['diffuse_fraction']
    
    df_work['dni'] = 0.0
    df_work.loc[daytime_mask, 'dni'] = np.clip(
        (df_work.loc[daytime_mask, 'ghi'].values - df_work.loc[daytime_mask, 'dhi'].values) / 
        (cos_zenith[daytime_mask] + 1e-10),
        0, None
    )
    
    print("Calculating POA irradiance...")
    
    # Transposition to tilted surface
    poa_irradiance = irradiance.get_total_irradiance(
        surface_tilt=pv_config['tilt'],
        surface_azimuth=pv_config['azimuth'],
        solar_zenith=zenith,
        solar_azimuth=azimuth,
        dni=df_work['dni'].values,
        ghi=df_work['ghi'].values,
        dhi=df_work['dhi'].values,
        model='isotropic'
    )
    
    # Extract POA components (returns dict with arrays)
    df_work['poa_global'] = poa_irradiance['poa_global']
    df_work['poa_direct'] = poa_irradiance['poa_direct']
    df_work['poa_diffuse'] = poa_irradiance['poa_diffuse']
    
    # Clip to physical limits
    df_work['poa_global'] = df_work['poa_global'].clip(lower=0, upper=1500)
    df_work['poa_direct'] = df_work['poa_direct'].clip(lower=0, upper=1200)
    df_work['poa_diffuse'] = df_work['poa_diffuse'].clip(lower=0, upper=500)
    
    print(f"\n{'='*70}")
    print(f"Irradiance conversion complete")
    print(f"Mean GHI: {df_work['ghi'].mean():.1f} W/m²")
    print(f"Mean POA: {df_work['poa_global'].mean():.1f} W/m²")
    print(f"POA/GHI ratio: {(df_work['poa_global'].mean() / (df_work['ghi'].mean() + 1e-10)):.3f}")
    print(f"{'='*70}\n")
    
    output_file = OUTPUT_DIR / 'step2_climate_with_poa_irradiance.csv'
    cols_to_save = ['datetime', 'year', 'month', 'hour', 'day_of_year', 'ssp',
                    'ghi', 'dni', 'dhi', 'poa_global', 'poa_direct', 'poa_diffuse',
                    'temperature_C', 'zenith']
    df_work[cols_to_save].to_csv(output_file, index=False)
    print(f"✓ Saved: {output_file}\n")
    
    return df_work

# =============================================================================
# KDE-BASED SYNTHETIC YEAR GENERATION
# =============================================================================

def generate_kde_synthetic_years(ssp_data, n_synthetic_years=100, perturbation_strength=0.2):
    """
    Generate synthetic years using conditional kernel density estimation.
    
    Strategy:
    1. Select a template year (closest to median annual irradiance)
    2. For each hour, sample from KDE trained on similar hours across all years
    3. Blend template values with KDE samples for smooth variation
    
    Parameters
    ----------
    ssp_data : pd.DataFrame
        Climate data for one SSP scenario
    n_synthetic_years : int
        Number of synthetic years to generate
    perturbation_strength : float
        Blend factor: 0=template only, 1=KDE only
        
    Returns
    -------
    list of pd.DataFrame
        Synthetic year dataframes
        
    Notes
    -----
    Conditional KDE preserves temporal correlations by sampling from
    similar times of day and year, avoiding physically unrealistic
    combinations (e.g., summer temperature with winter irradiance).
    """
    # Select template year (closest to median annual irradiance)
    annual_radiation = ssp_data.groupby('year')['poa_global'].mean()
    median_radiation = annual_radiation.median()
    
    closest_idx = (annual_radiation - median_radiation).abs().argmin()
    template_year = annual_radiation.index[closest_idx]
    
    template_data = ssp_data[ssp_data['year'] == template_year].copy().reset_index(drop=True)
    
    print(f"  Template year: {template_year} ({len(template_data)} hours)")
    
    synthetic_years = []
    
    for synth_idx in range(n_synthetic_years):
        synthetic_records = []
        
        for idx, template_row in template_data.iterrows():
            hour = template_row['hour']
            month = template_row['month']
            
            # Find similar hours (±1 hour window, same month)
            similar_mask = (
                (ssp_data['month'] == month) &
                (ssp_data['hour'] >= hour - 1) &
                (ssp_data['hour'] <= hour + 1)
            )
            similar_data = ssp_data[similar_mask][['ghi', 'poa_global', 'temperature_C']].values
            
            if len(similar_data) >= 10:
                try:
                    # Train KDE and sample
                    kde = gaussian_kde(similar_data.T)
                    sampled = kde.resample(1).T[0]
                    
                    # Blend template and KDE sample
                    new_ghi = perturbation_strength * template_row['ghi'] + (1 - perturbation_strength) * max(0, sampled[0])
                    new_poa = perturbation_strength * template_row['poa_global'] + (1 - perturbation_strength) * max(0, sampled[1])
                    new_temperature = perturbation_strength * template_row['temperature_C'] + (1 - perturbation_strength) * sampled[2]
                except:
                    # Fallback: random sample from similar data
                    rand_idx = np.random.randint(len(similar_data))
                    new_ghi = max(0, similar_data[rand_idx, 0])
                    new_poa = max(0, similar_data[rand_idx, 1])
                    new_temperature = similar_data[rand_idx, 2]
            else:
                # Not enough data: use template values
                new_ghi = template_row['ghi']
                new_poa = template_row['poa_global']
                new_temperature = template_row['temperature_C']
            
            synthetic_records.append({
                'datetime': template_row['datetime'],
                'year': template_year,
                'month': month,
                'hour': hour,
                'day_of_year': template_row['day_of_year'],
                'ssp': template_row['ssp'],
                'ghi': new_ghi,
                'poa_global': new_poa,
                'temperature_C': new_temperature,
                'synthetic_id': synth_idx
            })
        
        synth_year_df = pd.DataFrame(synthetic_records)
        synthetic_years.append(synth_year_df)
        
        if (synth_idx + 1) % 20 == 0:
            print(f"    Generated {synth_idx + 1}/{n_synthetic_years}...")
    
    return synthetic_years

def select_quantile_scenarios_from_synthetic(synthetic_years, quantiles=[0.1, 0.5, 0.9]):
    """
    Select representative scenarios using quantile-based sampling.
    
    Ranks synthetic years by annual mean POA irradiance and selects
    scenarios at specified quantiles (e.g., 10th, 50th, 90th percentile).
    
    Parameters
    ----------
    synthetic_years : list of pd.DataFrame
        Generated synthetic years
    quantiles : list of float
        Quantiles to select (0-1 scale)
        
    Returns
    -------
    pd.DataFrame
        Selected scenarios with quantile labels
        
    Notes
    -----
    Quantile selection provides:
    - Q10: Low-irradiance scenario (pessimistic)
    - Q50: Median scenario (expected)
    - Q90: High-irradiance scenario (optimistic)
    """
    # Calculate annual statistics for each synthetic year
    annual_stats = []
    for synth_df in synthetic_years:
        annual_stats.append({
            'synthetic_id': synth_df['synthetic_id'].iloc[0],
            'poa_mean': synth_df['poa_global'].mean(),
            'temp_mean': synth_df['temperature_C'].mean()
        })
    
    stats_df = pd.DataFrame(annual_stats)
    selected_scenarios = []
    
    for q in quantiles:
        # Find synthetic year closest to this quantile
        target_poa = stats_df['poa_mean'].quantile(q)
        closest_idx = (stats_df['poa_mean'] - target_poa).abs().idxmin()
        selected_id = stats_df.loc[closest_idx, 'synthetic_id']
        
        selected_year = synthetic_years[int(selected_id)].copy()
        selected_year['quantile'] = q
        selected_year['quantile_label'] = f"Q{int(q*100)}"
        selected_scenarios.append(selected_year)
        
        print(f"    Q{int(q*100)}: POA_mean={stats_df.loc[closest_idx, 'poa_mean']:.1f} W/m²")
    
    return pd.concat(selected_scenarios, ignore_index=True)

def generate_climate_scenarios(df, n_synthetic_per_ssp=100, quantiles=[0.1, 0.5, 0.9]):
    """
    Generate climate scenarios for all SSP pathways.
    
    Parameters
    ----------
    df : pd.DataFrame
        Climate data with POA irradiance
    n_synthetic_per_ssp : int
        Number of synthetic years per SSP
    quantiles : list of float
        Quantiles to select for final scenarios
        
    Returns
    -------
    pd.DataFrame
        Selected scenarios for all SSPs and quantiles
    """
    print(f"{'='*70}")
    print(f"STEP 3: KDE-BASED SCENARIO GENERATION")
    print(f"{'='*70}\n")
    
    ssp_list = sorted(df['ssp'].unique())
    all_scenarios = []
    
    for ssp in ssp_list:
        print(f"Processing {ssp}...")
        ssp_data = df[df['ssp'] == ssp].copy()
        
        synthetic_years = generate_kde_synthetic_years(
            ssp_data, 
            n_synthetic_years=n_synthetic_per_ssp,
            perturbation_strength=0.2
        )
        
        selected_scenarios = select_quantile_scenarios_from_synthetic(
            synthetic_years, 
            quantiles=quantiles
        )
        
        all_scenarios.append(selected_scenarios)
        print()
    
    result_df = pd.concat(all_scenarios, ignore_index=True)
    
    print(f"{'='*70}")
    print(f"Scenarios generated: {len(result_df):,} records")
    print(f"{'='*70}\n")
    
    output_file = OUTPUT_DIR / 'step3_climate_scenarios_kde_quantile.csv'
    result_df.to_csv(output_file, index=False)
    print(f"✓ Saved: {output_file}\n")
    
    return result_df

# =============================================================================
# PV POWER CALCULATION FOR SCENARIOS
# =============================================================================

def calculate_pv_power_for_scenarios(df, pv_config=PV_CONFIG):
    """
    Calculate PV power output for all scenarios.
    
    Parameters
    ----------
    df : pd.DataFrame
        Scenarios with POA irradiance and temperature
    pv_config : dict
        PV system configuration
        
    Returns
    -------
    pd.DataFrame
        Input data with 'pv_power_kw' column added
    """
    print(f"{'='*70}")
    print(f"STEP 4: CALCULATING PV POWER OUTPUT")
    print(f"{'='*70}\n")
    
    df['pv_power_kw'] = calculate_pv_power_flexible(
        temperature=df['temperature_C'].values,
        gti_irradiance=df['poa_global'].values,
        N_PV=pv_config['N_PV'],
        use_rated_power=False
    )
    
    # Print summary by SSP and quantile
    for ssp in sorted(df['ssp'].unique()):
        print(f"\n{ssp.upper()}:")
        for q_label in sorted(df[df['ssp'] == ssp]['quantile_label'].unique()):
            scenario_data = df[(df['ssp'] == ssp) & (df['quantile_label'] == q_label)]
            annual_energy = scenario_data['pv_power_kw'].sum()
            
            # Calculate capacity factor
            rated_capacity_kw = pv_config['N_PV'] * pv_config['module_specs']['ratedpowerkw']
            cf = (annual_energy / (rated_capacity_kw * 8760)) * 100
            
            print(f"  {q_label}: {annual_energy:,.0f} kWh/year, CF={cf:.2f}%")
    
    print(f"\n{'='*70}\n")
    
    output_file = OUTPUT_DIR / 'step4_scenarios_with_pv_power.csv'
    df.to_csv(output_file, index=False)
    print(f"✓ Saved: {output_file}\n")
    
    return df

# =============================================================================
# TECHNOLOGY SENSITIVITY ANALYSIS
# =============================================================================

def technology_sensitivity_analysis(df_scenarios, pv_config=PV_CONFIG):
    """
    Analyze impact of PV panel efficiency improvements on power generation.
    
    Separates climate uncertainty from technology uncertainty by fixing
    climate scenarios and varying panel efficiency.
    
    Parameters
    ----------
    df_scenarios : pd.DataFrame
        Climate scenarios with POA irradiance
    pv_config : dict
        PV system configuration
        
    Returns
    -------
    tech_results_df : pd.DataFrame
        Detailed results for all technology-climate combinations
    tech_summary : pd.DataFrame
        Summary statistics by technology scenario
        
    Notes
    -----
    Technology scenarios represent:
    - Current (2025): 20% efficiency (baseline)
    - Conservative (2030): 21.5% (+6.5%)
    - Moderate (2035): 23% (+14%)
    - Advanced (2040): 25% (+24%)
    - Future (2045): 27% (+34%, lab-achieved)
    """
    print(f"\n{'='*70}")
    print(f"STEP 5: TECHNOLOGY SENSITIVITY ANALYSIS")
    print(f"{'='*70}")
    print(f"Analyzing PV panel efficiency variations\n")
    
    # Define efficiency scenarios
    efficiency_scenarios = {
        'Current_2025': 0.20,        # 20% efficiency (baseline)
        'Conservative_2030': 0.215,   # +7.5% improvement
        'Moderate_2035': 0.23,        # +15% improvement
        'Advanced_2040': 0.25,        # +25% improvement
        'Future_2045': 0.27          # +35% improvement
    }
    
    print("Technology Scenarios:")
    print("-" * 70)
    for tech_name, efficiency in efficiency_scenarios.items():
        print(f"  {tech_name:20s}: {efficiency*100:.2f}% efficiency")
    print()
    
    all_tech_results = []
    
    for ssp in sorted(df_scenarios['ssp'].unique()):
        for q_label in sorted(df_scenarios[df_scenarios['ssp'] == ssp]['quantile_label'].unique()):
            
            # Get climate scenario data
            scenario_data = df_scenarios[
                (df_scenarios['ssp'] == ssp) & 
                (df_scenarios['quantile_label'] == q_label)
            ].copy()
            
            climate_scenario_id = f"{ssp}_{q_label}"
            
            # Calculate power for each efficiency level
            for tech_name, efficiency in efficiency_scenarios.items():
                
                # Modify module specs with new efficiency
                modified_specs = pv_config['module_specs'].copy()
                modified_specs['panel_efficiency'] = efficiency
                
                power_kw = calculate_pv_power_flexible(
                    temperature=scenario_data['temperature_C'].values,
                    gti_irradiance=scenario_data['poa_global'].values,
                    N_PV=pv_config['N_PV'],
                    module_specs=modified_specs,
                    use_rated_power=False
                )
                
                annual_energy = power_kw.sum()
                rated_capacity = pv_config['N_PV'] * pv_config['module_specs']['ratedpowerkw']
                capacity_factor = (annual_energy / (rated_capacity * 8760)) * 100
                
                # Calculate improvement vs baseline
                baseline_efficiency = 0.20
                relative_improvement = ((efficiency - baseline_efficiency) / baseline_efficiency) * 100
                
                all_tech_results.append({
                    'climate_scenario': climate_scenario_id,
                    'ssp': ssp,
                    'climate_quantile': q_label,
                    'technology': tech_name,
                    'efficiency_%': efficiency * 100,
                    'efficiency_improvement_%': relative_improvement,
                    'annual_energy_kWh': annual_energy,
                    'capacity_factor_%': capacity_factor,
                    'mean_power_kW': power_kw.mean(),
                    'peak_power_kW': power_kw.max()
                })
    
    tech_results_df = pd.DataFrame(all_tech_results)
    
    # Calculate summary statistics
    print("\n" + "="*70)
    print("TECHNOLOGY SENSITIVITY SUMMARY")
    print("="*70 + "\n")
    
    tech_summary = tech_results_df.groupby('technology').agg({
        'efficiency_%': 'first',
        'efficiency_improvement_%': 'first',
        'annual_energy_kWh': 'mean',
        'capacity_factor_%': 'mean'
    }).round(2)
    
    # Add energy gain metrics
    baseline_energy = tech_summary.loc['Current_2025', 'annual_energy_kWh']
    tech_summary['energy_gain_kWh'] = (tech_summary['annual_energy_kWh'] - baseline_energy).round(0)
    tech_summary['energy_gain_%'] = ((tech_summary['annual_energy_kWh'] / baseline_energy - 1) * 100).round(2)
    
    print("Average Performance Across All Climate Scenarios:")
    print(tech_summary)
    print()
    
    # Save results
    output_file = OUTPUT_DIR / 'step5_technology_sensitivity_detailed.csv'
    tech_results_df.to_csv(output_file, index=False)
    print(f"✓ Saved detailed results: {output_file}")
    
    output_file_summary = OUTPUT_DIR / 'step5_technology_sensitivity_summary.csv'
    tech_summary.to_csv(output_file_summary)
    print(f"✓ Saved summary: {output_file_summary}\n")
    
    # Generate comparison analysis
    create_technology_climate_comparison(tech_results_df)
    
    return tech_results_df, tech_summary

def create_technology_climate_comparison(tech_results_df):
    """
    Compare relative importance of climate vs technology uncertainty.
    
    Parameters
    ----------
    tech_results_df : pd.DataFrame
        Detailed technology sensitivity results
    """
    print("="*70)
    print("CLIMATE vs TECHNOLOGY IMPACT COMPARISON")
    print("="*70 + "\n")
    
    # 1. Climate impact (fixed technology = baseline)
    baseline_tech = tech_results_df[tech_results_df['technology'] == 'Current_2025']
    climate_variance = baseline_tech.groupby('ssp')['annual_energy_kWh'].agg(['min', 'max', 'mean'])
    climate_variance['range_%'] = ((climate_variance['max'] - climate_variance['min']) / climate_variance['mean'] * 100).round(2)
    
    print("Climate Impact Analysis (Fixed Technology = Current 2025):")
    print(climate_variance)
    print(f"\nAverage climate-induced variation: ±{climate_variance['range_%'].mean():.2f}%")
    
    # 2. Technology impact (averaged across climates)
    tech_variance = tech_results_df.groupby('technology')['annual_energy_kWh'].mean()
    tech_range = ((tech_variance.max() - tech_variance.min()) / tech_variance.min() * 100)
    
    print(f"\n" + "="*70)
    print(f"Technology Impact Range: {tech_range:.2f}%")
    print(f"  From {tech_variance.min():.0f} kWh/year (Current)")
    print(f"  To {tech_variance.max():.0f} kWh/year (Future)")
    print("="*70 + "\n")
    
    # 3. Relative importance
    avg_climate_range = climate_variance['range_%'].mean()
    
    print("="*70)
    print("KEY FINDING: RELATIVE IMPORTANCE")
    print("="*70)
    print(f"Climate uncertainty range: ±{avg_climate_range:.2f}%")
    print(f"Technology improvement potential: +{tech_range:.2f}%")
    
    if tech_range > avg_climate_range * 2:
        conclusion = "Technology improvements have MUCH GREATER impact than climate uncertainty"
    elif tech_range > avg_climate_range:
        conclusion = "Technology improvements have GREATER impact than climate uncertainty"
    else:
        conclusion = "Climate uncertainty is COMPARABLE TO or LARGER than technology impact"
    
    print(f"\nConclusion: {conclusion}")
    print("="*70 + "\n")
    
    # Save comparison
    comparison_data = {
        'Factor': ['Climate Variation (Current Tech)', 'Technology Improvement (All Climates)'],
        'Impact_%': [avg_climate_range, tech_range],
        'Type': ['Uncertainty', 'Opportunity']
    }
    comparison_df = pd.DataFrame(comparison_data)
    
    output_file = OUTPUT_DIR / 'step5_climate_vs_technology_comparison.csv'
    comparison_df.to_csv(output_file, index=False)
    print(f"✓ Saved comparison: {output_file}\n")

# =============================================================================
# SUMMARY STATISTICS
# =============================================================================

def generate_summary_statistics(df, pv_config=PV_CONFIG):
    """
    Generate annual and monthly summary statistics.
    
    Parameters
    ----------
    df : pd.DataFrame
        Scenarios with PV power output
    pv_config : dict
        PV system configuration
        
    Returns
    -------
    annual_summary : pd.DataFrame
        Annual statistics by SSP and quantile
    monthly_summary : pd.DataFrame
        Monthly statistics by SSP and quantile
    """
    print(f"{'='*70}")
    print(f"STEP 6: GENERATING SUMMARY STATISTICS")
    print(f"{'='*70}\n")
    
    # Annual summary
    annual_summary = df.groupby(['ssp', 'quantile_label']).agg({
        'pv_power_kw': ['sum', 'mean', 'max'],
        'poa_global': 'mean',
        'temperature_C': 'mean'
    }).round(2)
    
    annual_summary.columns = ['Annual_kWh', 'Mean_Power_kW', 'Peak_Power_kW',
                              'Mean_POA_W/m2', 'Mean_Temp_C']
    
    rated_capacity = pv_config['N_PV'] * pv_config['module_specs']['ratedpowerkw']
    annual_summary['Capacity_Factor_%'] = (
        (annual_summary['Annual_kWh'] / (rated_capacity * 8760)) * 100
    ).round(2)
    
    print(annual_summary)
    
    output_file = OUTPUT_DIR / 'step6_annual_summary_statistics.csv'
    annual_summary.to_csv(output_file)
    print(f"\n✓ Saved: {output_file}\n")
    
    # Monthly summary
    df['month_name'] = pd.to_datetime(df['datetime']).dt.strftime('%B')
    monthly_summary = df.groupby(['ssp', 'quantile_label', 'month', 'month_name']).agg({
        'pv_power_kw': ['sum', 'mean'],
        'poa_global': 'mean'
    }).round(2)
    
    output_file = OUTPUT_DIR / 'step6_monthly_summary_statistics.csv'
    monthly_summary.to_csv(output_file)
    print(f"✓ Saved: {output_file}\n")
    
    return annual_summary, monthly_summary

# =============================================================================
# MAIN PIPELINE
# =============================================================================

def main():
    """
    Run complete pipeline: SSP data → PV scenarios → Technology sensitivity.
    
    This pipeline requires:
    - SSP climate projection data (CSV files with rsds, tas columns)
    - Site configuration (lat/lon, altitude, timezone)
    - PV system configuration (tilt, azimuth, module specs)
    """

    # USER MUST SPECIFY: Path to SSP climate data directory
    data_path = './climate_data'  # Modify this path 
    
    print("\n" + "="*70)
    print("CLIMATE SCENARIOS → PV POWER PIPELINE")
    print("="*70 + "\n")
    
    # Step 1: Load SSP climate data
    df_climate = load_ssp_climate_data(data_path)
    
    # Step 2: Convert to tilted irradiance
    df_with_poa = convert_to_tilted_irradiance_pvlib(df_climate)
    
    # Step 3: Generate scenarios using KDE
    df_scenarios = generate_climate_scenarios(
        df_with_poa,
        n_synthetic_per_ssp=100,
        quantiles=[0.1, 0.5, 0.9]
    )
    
    # Step 4: Calculate PV power
    df_with_power = calculate_pv_power_for_scenarios(df_scenarios)
    
    # Step 5: Technology sensitivity analysis
    tech_results, tech_summary = technology_sensitivity_analysis(df_scenarios)
    
    # Step 6: Summary statistics
    annual_summary, monthly_summary = generate_summary_statistics(df_with_power)
    
    print("\n" + "="*70)
    print("PIPELINE COMPLETE!")
    print("="*70)
    print(f"\nAll outputs saved to: {OUTPUT_DIR}")
    print("\nGenerated files:")
    print("  1. step1_raw_climate_data.csv")
    print("  2. step2_climate_with_poa_irradiance.csv")
    print("  3. step3_climate_scenarios_kde_quantile.csv")
    print("  4. step4_scenarios_with_pv_power.csv")
    print("  5. step5_technology_sensitivity_detailed.csv")
    print("  6. step5_technology_sensitivity_summary.csv")
    print("  7. step5_climate_vs_technology_comparison.csv")
    print("  8. step6_annual_summary_statistics.csv")
    print("  9. step6_monthly_summary_statistics.csv")
    print("="*70 + "\n")

if __name__ == "__main__":
    main()
